# EB5002 Text Processing

# Data Exploration

## Set the working directory

In [1]:
import os
os.chdir(os.path.join(os.path.sep, 'home', 'tkokkeng', 'Documents', 'EB5002-TextProcessing'))
os.getcwd()

'/home/tkokkeng/Documents/EB5002-TextProcessing'

In [2]:
import sys
if os.path.join(os.getcwd(), 'source') not in sys.path:
    sys.path.append(os.path.join(os.getcwd(), 'source'))
sys.path

['/home/tkokkeng/Documents/EB5002-TextProcessing',
 '/home/tkokkeng/Documents/EB5002-TextProcessing/source',
 '/home/tkokkeng/python/python367/ptvenv/lib/python36.zip',
 '/home/tkokkeng/python/python367/ptvenv/lib/python3.6',
 '/home/tkokkeng/python/python367/ptvenv/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '',
 '/home/tkokkeng/python/python367/ptvenv/lib/python3.6/site-packages',
 '/home/tkokkeng/.local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/home/tkokkeng/.local/lib/python3.6/site-packages/IPython/extensions',
 '/home/tkokkeng/.ipython']

## Import libraries

In [43]:
import pandas as pd
import re
import io
import json

## Load data

In [4]:
if os.path.isfile(os.path.join('data', 'tweets.csv')):
    tweets_df = pd.read_csv(os.path.join('data', 'tweets.csv'), encoding='utf-8')
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6444 entries, 0 to 6443
Data columns (total 28 columns):
id                         6444 non-null int64
handle                     6444 non-null object
text                       6444 non-null object
is_retweet                 6444 non-null bool
original_author            722 non-null object
time                       6444 non-null object
in_reply_to_screen_name    208 non-null object
in_reply_to_status_id      202 non-null float64
in_reply_to_user_id        208 non-null float64
is_quote_status            6444 non-null bool
lang                       6444 non-null object
retweet_count              6444 non-null int64
favorite_count             6444 non-null int64
longitude                  12 non-null float64
latitude                   12 non-null float64
place_id                   204 non-null object
place_full_name            204 non-null object
place_name                 204 non-null object
place_type                 204 non-null obj

In [5]:
max(map(len, tweets_df['text']))

148

In [6]:
pd.set_option('max_colwidth', 200)

In [7]:
tweets_df['text'][:15]

0                        The question in this election: Who can put the plans into action that will make your life better? https://t.co/XreEY9OicG
1                        Last night, Donald Trump said not paying taxes was "smart." You know what I call it? Unpatriotic. https://t.co/t0xmBfj7zF
2      Couldn't be more proud of @HillaryClinton. Her vision and command during last night's debate showed that she's ready to be our next @POTUS.
3           If we stand together, there's nothing we can't do. \n\nMake sure you're ready to vote: https://t.co/tTgeqxNqYm https://t.co/Q3Ymbb7UNy
4                       Both candidates were asked about how they'd confront racial injustice. Only one had a real answer. https://t.co/sjnEokckis
5                          Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc
6       This election is too important to sit out. Go to https://t.co/tTgeqxNqYm and make sure you're registered. #Nat

## Tokenise the tweets

In [8]:
try: # Use the default NLTK tokenizer.
    from nltk import word_tokenize, sent_tokenize 
    # Testing whether it works. 
    # Sometimes it doesn't work on some machines because of setup issues.
    word_tokenize(sent_tokenize("This is a foobar sentence. Yes it is.")[0])
except: # Use a naive sentence tokenizer and toktok.
    import re
    from nltk.tokenize import ToktokTokenizer
    # See https://stackoverflow.com/a/25736515/610569
    sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
    # Use the toktok tokenizer that requires no dependencies.
    toktok = ToktokTokenizer()
    word_tokenize = toktok.tokenize

In [30]:
tokenised_tweets = tweets_df['text'].map(lambda x: [ list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(x) ])

In [31]:
tokenised_tweets[:15]

0                                         [[the, question, in, this, election, :, who, can, put, the, plans, into, action, that, will, make, your, life, better, ?, https://t.co/xreey9oicg]]
1                                    [[last, night, ,, donald, trump, said, not, paying, taxes, was, ", smart., ", you, know, what, i, call, it, ?], [unpatriotic., https://t.co/t0xmbfj7zf]]
2     [[couldn, ', t, be, more, proud, of, @hillaryclinton, .], [her, vision, and, command, during, last, night, ', s, debate, showed, that, she, ', s, ready, to, be, our, next, @potus, .]]
3                      [[if, we, stand, together, ,, there, ', s, nothing, we, can, ', t, do., make, sure, you, ', re, ready, to, vote, :, https://t.co/ttgeqxnqym, https://t.co/q3ymbb7uny]]
4                                      [[both, candidates, were, asked, about, how, they, ', d, confront, racial, injustice, .], [only, one, had, a, real, answer., https://t.co/sjneokckis]]
5                                        [[join, m

In [32]:
tokenised_tweets = tokenised_tweets.sum()

In [33]:
len(tokenised_tweets)

9236

In [34]:
tokenised_tweets[:5]

[['the',
  'question',
  'in',
  'this',
  'election',
  ':',
  'who',
  'can',
  'put',
  'the',
  'plans',
  'into',
  'action',
  'that',
  'will',
  'make',
  'your',
  'life',
  'better',
  '?',
  'https://t.co/xreey9oicg'],
 ['last',
  'night',
  ',',
  'donald',
  'trump',
  'said',
  'not',
  'paying',
  'taxes',
  'was',
  '"',
  'smart.',
  '"',
  'you',
  'know',
  'what',
  'i',
  'call',
  'it',
  '?'],
 ['unpatriotic.', 'https://t.co/t0xmbfj7zf'],
 ['couldn', "'", 't', 'be', 'more', 'proud', 'of', '@hillaryclinton', '.'],
 ['her',
  'vision',
  'and',
  'command',
  'during',
  'last',
  'night',
  "'",
  's',
  'debate',
  'showed',
  'that',
  'she',
  "'",
  's',
  'ready',
  'to',
  'be',
  'our',
  'next',
  '@potus',
  '.']]

## Remove all urls from the tweets

In [35]:
tokenised_tweets = [ list(filter(lambda x: not re.match('https://', x), sent)) for sent in tokenised_tweets ]

In [36]:
tokenised_tweets[:5]

[['the',
  'question',
  'in',
  'this',
  'election',
  ':',
  'who',
  'can',
  'put',
  'the',
  'plans',
  'into',
  'action',
  'that',
  'will',
  'make',
  'your',
  'life',
  'better',
  '?'],
 ['last',
  'night',
  ',',
  'donald',
  'trump',
  'said',
  'not',
  'paying',
  'taxes',
  'was',
  '"',
  'smart.',
  '"',
  'you',
  'know',
  'what',
  'i',
  'call',
  'it',
  '?'],
 ['unpatriotic.'],
 ['couldn', "'", 't', 'be', 'more', 'proud', 'of', '@hillaryclinton', '.'],
 ['her',
  'vision',
  'and',
  'command',
  'during',
  'last',
  'night',
  "'",
  's',
  'debate',
  'showed',
  'that',
  'she',
  "'",
  's',
  'ready',
  'to',
  'be',
  'our',
  'next',
  '@potus',
  '.']]

## Save the tokenised tweets

In [44]:
with io.open(os.path.join('data', 'tokenised-tweets.txt'), 'w', encoding='utf8') as outfile:
    json.dump(tokenised_tweets, outfile, ensure_ascii=False)